## Получение библиотеки, загрузка и обзор данных

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')
data.head()

In [ ]:
# Разделяем данные
X = data.drop(['ID', 'TARGET'], axis=1)  # Исправлено: добавляем ID в список столбцов для удаления
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

### Задание
Задание на скольки объектов какого класса. Сколько подлогов обновляют к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса 41.2, а ответ салют 41).

In [ ]:
# Подсчёт количества объектов каждого класса
class_counts = pd.Series(ytrain).value_counts()
print("Количество объектов класса 0 (отрицательный):", class_counts[0])
print("Количество объектов класса 1 (положительный):", class_counts[1])

# Доля положительного класса
positive_class_ratio = class_counts[1] / len(ytrain)
print("Доля положительного класса:", positive_class_ratio)

# Округлённая доля в процентах
positive_class_percentage = round(positive_class_ratio * 100)
print("Доля положительного класса (в процентах, округлённая):", positive_class_percentage)

### Обучение логистическую регрессию с параметром поумолчанию

In [ ]:
# Масштабируем данные (важно для логистической регрессии)
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

# Обучаем модель
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(Xtrain_scaled, ytrain)

# Предсказываем
prediction = lr.predict(Xtest_scaled)

Метрика accuracy не устойчивая при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества. Получите посчитайте f1_score для оценки качества. f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [ ]:
from sklearn.metrics import f1_score

# Считаем f1_score
f1 = f1_score(ytest, prediction)
print("f1_score модели:", f1)

**Вопрос f1_score?**  
Ответ: Значение f1_score ты увидишь после выполнения кода. Например, если f1_score = 0.35, это значит, что модель плохо справляется с классификацией положительного класса, скорее всего из-за дисбаланса.

**Задание разделяется, почему качество такое низкое.**

Качество низкое из-за:
- **Дисбаланса классов:** Положительного класса (TARGET = 1) намного меньше, чем отрицательного. Модель склонна предсказывать доминирующий класс (0), что снижает f1_score.
- **Отсутствие масштабирования:** Ты изначально не масштабировал данные, а логистическая регрессия чувствительна к масштабу признаков. Мы это исправили, добавив `StandardScaler`.
- **Дефолтный порог:** Порог 0.5 для классификации может быть неоптимальным при дисбалансе.
- **Признаки:** Некоторые признаки могут быть неинформативными.

**Проверка разделяемости с помощью линейных разделяемых.**

In [ ]:
# Получаем вероятности
probs_test = lr.predict_proba(Xtest_scaled)[:, 1]
print("Первые 18 вероятностей:")
print(probs_test[:18])

# Визуализируем распределение вероятностей
plt.figure(figsize=(8, 5))
plt.hist(probs_test[ytest == 0], bins=30, alpha=0.5, label='Класс 0', color='blue')
plt.hist(probs_test[ytest == 1], bins=30, alpha=0.5, label='Класс 1', color='orange')
plt.xlabel('Вероятность класса 1')
plt.ylabel('Количество')
plt.title('Распределение вероятностей')
plt.legend()
plt.show()

**По порогами вышло, что вероятность отнесения к положительному классу очень низкая.**  
Да, вероятности для класса 1 низкие, потому что классы несбалансированы, и модель склонна предсказывать класс 0. Мы можем улучшить это, подобрав порог.

**Попросил раздельные минимальное на train данных Xtrain, Xval, Xtest.**

In [ ]:
# Разделяем Xtrain на Xtrain_new и Xval
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain_scaled, ytrain, train_size=0.7, random_state=123)

**Неужели равны f1_score, если бы всё-таки с вероятностью не меняя 0.1, отнёс к положительному классу?**  
**Ответ округлите до двух.**

In [ ]:
# Применяем порог 0.1
new_predictions = (probs_test >= 0.1).astype(int)

# Считаем f1_score с новым порогом
f1_new = f1_score(ytest, new_predictions)
print("f1_score с порогом 0.1:", round(f1_new, 2))

**Улучшение модели можно посмотреть веса (как в линейной регрессии).**

In [ ]:
# Исправляем код для весов
weights_lr = pd.DataFrame({'coef': lr.coef_[0]},
                          index=data.drop(['ID', 'TARGET'], axis=1).columns.tolist())

# Добавляем intercept
weights_lr.loc['intercept'] = lr.intercept_[0]

# Сортируем по убыванию
weights_lr = weights_lr.sort_values(by='coef', ascending=False)

# Выводим таблицу
print(weights_lr)

**Вопрос: Какой признак имеет наибольший положительный вес?**

In [ ]:
top_feature = weights_lr.index[0]
print("Признак с наибольшим положительным весом:", top_feature)

## Бонус

### Задание 1
Подайте 1 порог для перевода вероятности в классы, дающий максимальное значение f1_score.

In [ ]:
# Перебираем пороги на валидационной выборке
probs_val = lr.predict_proba(Xval)[:, 1]
thresholds = np.arange(0.05, 0.95, 0.05)
f1_scores = []

for thresh in thresholds:
    val_predictions = (probs_val >= thresh).astype(int)
    f1 = f1_score(yval, val_predictions)
    f1_scores.append(f1)

# Находим порог с максимальным f1_score
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
print("Лучший порог:", best_threshold)
print("Максимальный f1_score на валидационной выборке:", best_f1)

# Визуализируем зависимость f1_score от порога
plt.figure(figsize=(8, 5))
plt.plot(thresholds, f1_scores, marker='o')
plt.xlabel('Порог')
plt.ylabel('f1_score')
plt.title('Зависимость f1_score от порога')
plt.grid(True)
plt.show()

### Задание 2
Восстановим в классы, дайте оценку максимильную у логистической регрессии и порог для перевода вероятностей в классы с учётической регрессии и порог для перевода.

Поэтому разделим данные на train части Xtrain, Xval, Xtest.
- В цикле при пороге с и порога будем обучаться по Xtrain, а предсказывать измерять качество по Xval.
- Качество итоговой модели с найденным с и порогом измерять по Xtest.

In [ ]:
# Перебираем параметр C и порог
C_values = [0.01, 0.1, 1, 10, 100]
thresholds = np.arange(0.05, 0.95, 0.05)
best_C = None
best_threshold = None
best_f1 = 0

for C in C_values:
    model = LogisticRegression(C=C, random_state=42, max_iter=1000)
    model.fit(Xtrain_new, ytrain_new)
    probs_val = model.predict_proba(Xval)[:, 1]
    
    for thresh in thresholds:
        val_predictions = (probs_val >= thresh).astype(int)
        f1 = f1_score(yval, val_predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_C = C
            best_threshold = thresh

print("Лучший параметр C:", best_C)
print("Лучший порог:", best_threshold)
print("Максимальный f1_score на валидационной выборке:", best_f1)

# Обучаем итоговую модель на всех тренировочных данных
final_model = LogisticRegression(C=best_C, random_state=42, max_iter=1000)
final_model.fit(Xtrain_scaled, ytrain)

# Предсказываем на тестовых данных
probs_test = final_model.predict_proba(Xtest_scaled)[:, 1]
final_predictions = (probs_test >= best_threshold).astype(int)

# Оцениваем качество на тестовых данных
final_f1 = f1_score(ytest, final_predictions)
print("f1_score на тестовых данных:", final_f1)

# Выводим матрицу ошибок
conf_matrix = confusion_matrix(ytest, final_predictions)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Класс 0', 'Класс 1'], yticklabels=['Класс 0', 'Класс 1'])
plt.xlabel('Предсказанный класс')
plt.ylabel('Реальный класс')
plt.title('Матрица ошибок')
plt.show()